In [ ]:
%matplotlib inline
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import datetime
import cx_Oracle

#Get last month's data from the Oracle Database
dsn_tns    = cx_Oracle.makedsn("y27prd01.isd.upmc.edu",1521, "SIMP2")
# connection = cx_Oracle.connect('simmedical_prod', 'PROD7892', "y27prd01.isd.upmc.edu:1521/SIMP2")
connection = cx_Oracle.connect('simmedical_prod', 'PROD7892', dsn_tns)
selectStr = """select c.ABBRV COURSE, c.COURSE_ID, to_char(l.CLASS_DATE, 'YYYY-MM-DD') CLASS_DATE,
       l.CLASS_ID, a.EVAL_ANSWER_ID SCORE, to_char(m.EVALUATE_DATE, 'YYYY-MM-DD HH24:MI:SS') EVALUATE_DATE,
       case when instr(q.QUESTION_TEXT, 'Effectiveness') > 1 then 'E'
            when instr(q.QUESTION_TEXT, 'Likeliness')    > 1 then 'L'
       else 'F' END "TYPE" --Facilitator
  from EVALUATION_ANSWERS a, EVALUATION_MAIN m, CLASSES l, courses c, ID0_EVAL_QUESTIONS q
 where a.EVALUATION_ID = m.EVALUATION_ID
   and a.EVAL_QUESTION_ID = q.EVAL_QUESTION_ID
   and m.CLASS_ID = l.CLASS_ID
   and l.COURSE_ID = c.COURSE_ID
   -- This will pull the data from last month (if you run it before the 28th of the month).
   and trunc(m.EVALUATE_DATE) between to_date('2019-07-01', 'YYYY-MM-DD')
                                  and trunc(sysdate,    'mm')-1
--   and trunc(m.EVALUATE_DATE) < to_date('2020-05-01', 'YYYY-MM-DD')
   -- Effectiveness of the in-person education you received
   and (a.EVAL_QUESTION_ID in (36277, 36336, 36348, 36613, 36858, 36927, 37139, 37180, 37217, 37240, 37284, 37406, 37418, 37724, 37966, 38069, 38108, 38220)
       -- Likeliness of recommending this course to a colleague
        or a.EVAL_QUESTION_ID in (36855, 36924, 37136, 37176, 37192, 37194, 37214, 37237, 37281, 37299, 37403, 37415, 37721, 37831, 37951, 37963, 38066, 38105, 38219)
      -- The facilitator(s) made the educational experience relevant to my training level
        or a.EVAL_QUESTION_ID in (36280, 36339, 36351, 36616, 36861, 36930, 37142, 37183, 37220, 37244, 37287, 37409, 37421, 37727, 37969, 38072, 38111, 38223)
        )
order by EVALUATE_DATE"""
lastDF = pd.read_sql_query(selectStr, con=connection, parse_dates={'CLASS_DATE','EVALUATE_DATE'})
print(lastDF.info())
print(lastDF.head(20))